In [6]:
# reload during dev
from importlib import reload
import PET_raw_data
PET_raw_data = reload(PET_raw_data)



dict_load = {'[Adt Income] adult.csv'      : {'describe_params' : {'missing_level' : {'data': {'?'}}}}
            ,'[Census] adult.data'         : {'read_params'     : {'label_encoding' : 'Y'
                                                                  ,'header_exist' : 'N'
                                                                  ,'header'       : ['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','income']
                                                                  }
                                             ,'describe_params' : {'missing_level' : {'data': {'?'}}}
                                             }
            ,'[NHANES] B.csv'              : {'read_params'     : {'label_encoding' : 'Y'
                                                                  ,'header_exist' : 'N'
                                                                  ,'header'       : ['gen','age','race','edu','mar','bmi','dep','pir','gh','mets','qm','dia']
                                                                  }
                                             }
            ,'[Retail] Online Retail.xlsx' : {'read_method'     : 'pandas_xlsx'
                                             ,'read_params'     : {'label_encoding' : 'Y'
                                                                  ,'sheet_name' : 'Online Retail'
                                                                  ,'str_col'    : ['InvoiceNo','StockCode','InvoiceDate','CustomerID']
                                                                  }
                                             }
            ,'[UK-US PF] va_household_ver_1_9_instance_2.csv' : {'read_params' : {'label_encoding' : 'Y'
                                                                                 ,'str_col' : ['hid','rlid','admin1','admin2']}}
            ,'[UK-US PF] va_person_ver_1_9_instance_2.csv'    : {'read_params' : {'label_encoding' : 'Y'
                                                                                 ,'str_col' : ['hid','pid','person_number','sex']}}
            }
dict_out  = {'[Adt Income] adult.csv'      : "[Adt Income]"
            ,'[Census] adult.data'         : "[Census]"
            ,'[NHANES] B.csv'              : "[NHANES]"
            ,'[Retail] Online Retail.xlsx' : "[Retail]"
            ,'[UK-US PF] va_household_ver_1_9_instance_2.csv' : "[UK-US PF] Household"
            ,'[UK-US PF] va_person_ver_1_9_instance_2.csv'    : "[UK-US PF] Person"
            }



import time
from datetime import datetime
import pytz
from sdv.metadata import SingleTableMetadata
from sdv.single_table import GaussianCopulaSynthesizer ,CTGANSynthesizer ,TVAESynthesizer ,CopulaGANSynthesizer
n_sample = 10
library_name = 'SDV'


__temp = {}
for filename ,file_params in dict_load.items():
    file_params_init = {'read_params'     : {'label_encoding' : 'Y'}
                       ,'describe'        : 'N'
                       ,'describe_params' : {'correlation'   : 'N'
                                            ,'collinearity'  : 'N'}
                       ,'report'          : 'N'
                       ,'report_params'   : {'print_report' : 'N'
                                            ,'save_report'  : 'Y'}
                       }
    file_params_init.update(file_params)
    print(f"Now is {filename}")
    __temp[filename] = PET_raw_data.load_data(f".\data\{filename}" ,file_params_init)



    time_start = time.time()
    __data         = __temp[filename].data.copy()
    __data_row_num = __data.shape[0]
    metadata       = SingleTableMetadata()
    metadata.detect_from_dataframe(__data)
    ##
    __data_oricast   = __temp[filename].data_oricast.copy()
    metadata_oricast = SingleTableMetadata()
    metadata_oricast.detect_from_dataframe(__data)
    ##
    __data_label_encoding   = __temp[filename].data_label_encoding.copy()
    metadata_label_encoding = SingleTableMetadata()
    metadata_label_encoding.detect_from_dataframe(__data_label_encoding)
    print(f"讀 3*metafile 時間 {round(time.time()-time_start ,4)} 秒")

    for method_name in ['GaussianCoupula'
                       ,'CTGAN'
                       ,'TVAE'
                       ,'CoupulaGAN']:

        exectime = datetime.now().astimezone(pytz.timezone('Asia/Taipei'))
        exec_out = f"{dict_out[filename]}_[{library_name}]_[{method_name}]_{exectime.strftime('%Y%m%d_%H%M%S')}_{exectime.tzinfo.zone.replace('/' ,'_')}"

        print(f"現在套件是{method_name}")
        time_start = time.time()
        if   method_name == 'GaussianCoupula':
            synthesizer = GaussianCopulaSynthesizer(metadata)
            synthesizer.fit(__data)
        elif method_name == 'CTGAN':
            synthesizer = CTGANSynthesizer(metadata)
            synthesizer.fit(__data_oricast) # _label_encoding)
        elif method_name == 'TVAE':
            synthesizer = TVAESynthesizer(metadata)
            synthesizer.fit(__data_label_encoding)
        elif method_name == 'CoupulaGAN':
            synthesizer = CopulaGANSynthesizer(metadata)
            synthesizer.fit(__data_label_encoding)
        else:
            continue
        print(f"訓練時間 {round(time.time()-time_start ,4)} 秒")
        synthesizer.save(f".\model_dpsd\{exec_out}.pkl")


        time_start = time.time()
        for i in range(n_sample):
            synthesizer.reset_sampling()
            synthetic_data = synthesizer.sample(num_rows   = 10000
                                            ,batch_size = 1000
                                            )
            synthetic_data.to_csv(f".\data_dpsd\{exec_out}_10000.csv")
        print(f"合成 {n_sample} 次 10K 筆時間 {round(time.time()-time_start ,4)} 秒")

        time_start = time.time()
        synthetic_data = synthesizer.sample(num_rows   = __data_row_num
                                        ,batch_size = 1000
                                        )
        synthetic_data.to_csv(f".\data_dpsd\{exec_out}_{__data_row_num}.csv")
        print(f"合成一次同筆數時間 {round(time.time()-time_start ,4)} 秒")



        print(f"Done: {exec_out}")



        break

Now is [Adt Income] adult.csv
讀 3*metafile 時間 0.3033 秒
現在套件是GaussianCoupula
訓練時間 21.3066 秒


Sampling rows: 100%|██████████| 10000/10000 [00:02<00:00, 3546.09it/s]


合成 10 次 10K 筆時間 19.5777 秒


Sampling rows: : 49000it [00:08, 5904.62it/s]                         


合成一次同筆數時間 8.8564 秒
Done: [Adt Income]_[SDV]_[GaussianCoupula]_20231106_093205_Asia_Taipei
Now is [Census] adult.data


FileNotFoundError: [Errno 2] No such file or directory: '.\\data\\[Census] adult.data'